In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\acer\\OneDrive\\Desktop\\VIT MSD\\Project\\End to End ML ops along with deployment\\End-To-End-Machine-Learning-and-ML-flow-Project'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tushargoutam"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "b7cfe0b4e3d655802d2fe95cd35fd161485991a5"

In [3]:
import dagshub
dagshub.init(repo_owner='tushargoutam',
             repo_name='End-To-End-Machine-Learning-and-ML-flow-Project',
             mlflow=True)

Accessing as tushargoutam

Initialized MLflow to track repo "tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project"

Repository tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project initialized!

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [5]:
from src.Project.constants import *
from src.Project.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri = "https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow"
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [20]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local JSON file
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, artifact_path="model",)
            else:
                mlflow.sklearn.log_model(model, artifact_path="model")


In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-19 19:44:06,488 : INFO : common : yaml file : config\config.yaml loaded sucessfully]
[2025-10-19 19:44:06,489 : INFO : common : yaml file : params.yaml loaded sucessfully]
[2025-10-19 19:44:06,491 : INFO : common : yaml file : schema.yaml loaded sucessfully]
[2025-10-19 19:44:06,492 : INFO : common : Created directory at : artifacts]
[2025-10-19 19:44:06,494 : INFO : common : Created directory at : artifacts/model_evaluation]
[2025-10-19 19:44:07,640 : INFO : common : json file saved at : artifacts\model_evaluation\metrics.json]


2025/10/19 19:44:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run fearless-moth-179 at: https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow/#/experiments/0/runs/ba074eff183b4391ae93e0ddba30553a
🧪 View experiment at: https://dagshub.com/tushargoutam/End-To-End-Machine-Learning-and-ML-flow-Project.mlflow/#/experiments/0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\acer\OneDrive\Desktop\VIT MSD\Project\End to End ML ops along with deployment\End-To-End-Machine-Learning-and-ML-flow-Project\mlops\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_42192\4025126429.py", line 7, in <module>
    raise e
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_42192\4025126429.py", line 5, in <module>
    model_evaluation_config.log_into_mlflow()
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_42192\447965016.py", line 45, in log_into_mlflow
    mlflow.sklearn.log_model(model, artifact_path="model",)
  File "c:\Users\acer\OneDrive\Desktop\VIT MSD\Project\End to End ML ops along with deployment\End-To-End-Machine-Learning-and-ML-flow-Project\mlops\Lib\site-packages\mlflow\sklearn\__init__.py", line 426, in log_model
    signature=signature,
       ^^^^^^^^^^^^^^^^^
  File "c:\Us

In [16]:
pip uninstall -y mlflow


Found existing installation: mlflow 3.5.0
Uninstalling mlflow-3.5.0:
  Successfully uninstalled mlflow-3.5.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install mlflow==2.12.2


  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached markdown-3.9-py3-none-any.whl.metadata (5.1 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
   ---------------------------------------- 0.0/20.2 MB ? eta -:--:--
   -- ------------------------------------- 1.0/20.2 MB 10.1 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/20.2 MB 10.1 MB/s eta 0:00:02
   ---- ----------------------------------- 2.1/20.2 MB 3.9 MB/s eta 0:00:05
   ---- ----------------------------------- 2.1/20.2 MB 3.9 MB/s eta 0:00:05
   ----- ---------------------------------- 2.9/20.2 MB 2.7 MB/s eta 0:00:07
   ------ --------------------------------- 3.1/20.2 MB 2.9 MB/s eta 0:00:06
   -------- ------------------------------- 4.2/20.2 MB 3.0 MB/s eta 0:00:06
   -------- ------------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
